<a href="https://colab.research.google.com/github/ccnmagnoo/colabs/blob/main/sec_etl_professional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Resume
Actually there´s 11.700 profesional certificated in GAS and Electricity procedures. But Just 290 of them are women, and [SEC](www.sec.cl) (institution focused in regulate this profesionals), dosen´t focus in this labor. This code is used for extract and filter those women to use in diverse social programs, and visualice their labor and effort in this energy field

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import pandas as pd
SOURCE_WEB = 'https://wlhttp.sec.cl/buscadorinstaladores/buscador.do'
PATH_GDRIVE = '/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality'
eli_source = f'{PATH_GDRIVE}/sec_eli.xls'
gas_source = f'{PATH_GDRIVE}/sec_gas.xls'

#Preparing Dataset

In [31]:
#read gas & dataframe it
gas_df = pd.read_excel(gas_source,header=4)
gas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1898 entries, 0 to 1897
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              1898 non-null   object
 1   CORREO ELECTRÓNICO  1867 non-null   object
 2   RUT                 1898 non-null   object
 3   DIRECCIÓN           1898 non-null   object
 4   TELÉFONO            902 non-null    object
 5   CELULAR             1844 non-null   object
 6   LICENCIA CLASE      1898 non-null   object
dtypes: object(7)
memory usage: 103.9+ KB


In [32]:
#read electrica & dataframe it
eli_df = pd.read_excel(eli_source,header=4)
eli_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9881 entries, 0 to 9880
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              9881 non-null   object
 1   CORREO ELECTRÓNICO  9611 non-null   object
 2   RUT                 9881 non-null   object
 3   DIRECCIÓN           9881 non-null   object
 4   TELÉFONO            4465 non-null   object
 5   CELULAR             9614 non-null   object
 6   LICENCIA CLASE      9881 non-null   object
dtypes: object(7)
memory usage: 540.5+ KB


In [33]:
#joint gas + eli
joinAll = pd.concat([eli_df,gas_df])
joinAll.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11779 entries, 0 to 1897
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              11779 non-null  object
 1   CORREO ELECTRÓNICO  11478 non-null  object
 2   RUT                 11779 non-null  object
 3   DIRECCIÓN           11779 non-null  object
 4   TELÉFONO            5367 non-null   object
 5   CELULAR             11458 non-null  object
 6   LICENCIA CLASE      11779 non-null  object
dtypes: object(7)
memory usage: 736.2+ KB


#Definig Transform Functions

##Machine Learning functions

In [34]:
from numpy.lib.function_base import iterable
from pandas.io.formats.style_render import non_reducing_slice
from pandas.core.arrays import boolean
#name normalizer
from typing import Union, List
from enum import Enum
import numpy as np


#IMPORTANT : constant used in keras model version 2
CHARS:List[str] = list('aábcdeéfghiíjklmnñoópqrstuúvwxyz')
NAME_BLOCK_SIZE = 20

class Gender(Enum):
  MALE = 'male'
  FEMALE='female'

#normalize name for keras neuronal net consumption
def nameSerializer(unitName:str,size:int=20)->Union[List[float],None]:
  if unitName == None:
    return None

  name_array = list(unitName)
  # [f,i,r,s,t] ->[6/23,8/23,9/23,11/23,20/23] indexation
  descriteName = list(map(lambda char:CHARS.index(char)/len(CHARS),name_array))
  # content into a 20-size box [0.0 ,0.0 ,....0.0]
  box = [0.0]*(size-len(descriteName))
  input = np.concatenate((np.array(box),np.array(descriteName)))
  return list(input.flatten())


def nameNormalizer(input:str)->list[str]:
  cleaned = input.replace('-',' ')
  splited_name:list[str] = cleaned.lower().split(' ')
  splited_name = list(filter(lambda it: len(it)>1,splited_name))
  return splited_name

#import ML keras tensorflow model
import keras
from keras.optimizers import get
MODEL = 'keras_genderizer_model_v2.h5'
MODEL_PATH = f'/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/{MODEL}'
model = keras.models.load_model(MODEL_PATH)

#wrapping gender
def getGenderByKeras(unitName:str)->Gender|None:
  if unitName == None:
    return None

  options=[Gender.MALE,Gender.FEMALE]
  input = nameSerializer(unitName,20)

  prediction = model.predict([input])[0]
  category = options[np.argmax(prediction)]
  return category

#for saving process simplified selector with just "o" or "a"
def getGenderSimplified(unitName:str)->Gender|None:

  last_char = unitName[-1]
  if last_char == 'o':
    return Gender.MALE
  if last_char == 'a':
    return Gender.FEMALE
  return None

from pandas import DataFrame
WRONG_POSITIVE:DataFrame = pd.read_excel('/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/wrong_positive.xlsx')

def checkWrongPositive(unitName:str|None)-> bool:
  if not unitName:
    return False

  return WRONG_POSITIVE['nombre'].str.contains(unitName).any()

#main general function wrapper⭐
def getGender(name:str | None)->Gender| str | None:

  #checking input is empty
  if name ==  None:
    return None

  #catching wrong positive
  for word in nameNormalizer(name):
    if getGenderSimplified(word) == Gender.MALE:
      return Gender.MALE.value
      break
    if checkWrongPositive(word) == True:
      return Gender.MALE.value
      break

  #if simplified dosen´t work applys de keras model
  modeler  = getGenderByKeras(nameNormalizer(name)[0])
  if modeler:
    return modeler.value

  return None

In [35]:
#testing machine learning
getGender('Carlos - Campos')


1/1 [==============================] - 0s 154ms/step


'male'

#Transform Dataset & calculating columns

In [36]:
# Transform
## split NOMBRE column
joinAll[['surname','name']] = joinAll['NOMBRE'].str.split(',',1,expand=True)
## remove null names
joinAll['name'] = joinAll['name'].map(lambda name: name.strip())
joinAll['address'] = joinAll['DIRECCIÓN'].str.replace('Nunoa','Ñuñoa')

joinAll['movil'] = joinAll['CELULAR']
joinAll['movil'] = joinAll['movil'].str.replace(' ','')
joinAll['movil'] = joinAll['movil'].str.replace('-','')
joinAll['movil'] = pd.to_numeric(joinAll['movil'], errors='coerce').fillna(0).astype(int)
joinAll['movil'] = joinAll['movil'].apply(lambda c:normalizePhone(c))

joinAll['phone'] = joinAll['TELÉFONO']
joinAll['phone'] = joinAll['phone'].str.replace(' ','')
joinAll['phone'] = joinAll['phone'].str.replace('-','')
joinAll['phone'] = pd.to_numeric(joinAll['phone'], errors='coerce').fillna(0).astype(int)


joinAll.head(3)

<ipython-input-36-dedddc9d5402>:3: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  joinAll[['surname','name']] = joinAll['NOMBRE'].str.split(',',1,expand=True)


,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,surname,name,address,movil,phone
0,"Moraga Núñez , Jorge Orlando",jmoragan@gmail.com,7324296-7,"Cerro Altar 6616, 601, Las Condes",25774233,73381253,A,Moraga Núñez,Jorge Orlando,"Cerro Altar 6616, 601, Las Condes",5.697338e+10,25774233
1,"Sánchez Jorquera , Carlos Mauricio",carlos.msanchezj@gmail.com,18328350-2,"colina 4915, Conchalí",NaN,56940447001,A,Sánchez Jorquera,Carlos Mauricio,"colina 4915, Conchalí",5.694045e+10,0
2,"San Martín Salamanca , Felipe Andrés",felipe.sanmartin.salamanca@gmail.com,18101438-5,"Eleuterio Ramirez 1241, Pobl. 2 de Septiembre,...",NaN,934113388,B,San Martín Salamanca,Felipe Andrés,"Eleuterio Ramirez 1241, Pobl. 2 de Septiembre,...",5.693411e+10,0


##Detecting name gender

In [37]:
joinAll['gender'] = joinAll['name'].map(getGender)

1/1 [==============================] - 0s 99ms/step


In [38]:
#big filter
from pandas import DataFrame
#result:DataFrame = all_technicians.loc[all_technicians['gender']=='female']
result:DataFrame = joinAll
result.sample(10)

,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,surname,name,address,movil,phone,gender
525,"Romero Fuentealba , Boris Andrés",romeroboris@gmail.com,6846189-8,"ARTURO PRAT 1381, POBLACION NAVIDAD, Tomé",NaN,91502330,C,Romero Fuentealba,Boris Andrés,"ARTURO PRAT 1381, POBLACION NAVIDAD, Tomé",5.699150e+10,0,male
4857,"Olivares Cruz , Edwin Exequiel",eolivares@duinsoli.cl,14309631-9,"Berlin 2847, Pobl. Alemania, Calama",314894,76493025,A,Olivares Cruz,Edwin Exequiel,"Berlin 2847, Pobl. Alemania, Calama",5.697649e+10,314894,male
143,"Fuster Donoso , Iván Rodolfo",ivanfusterdonoso@gmail.com,13827223-0,"Tucapel 129, Los Andes",NaN,56984481628,A,Fuster Donoso,Iván Rodolfo,"Tucapel 129, Los Andes",5.698448e+10,0,male
3946,"Meléndez Moraga , Oscar Boris",oscarmelendez.moraga@gmail.com,15135349-5,"pasaje 22 norte a Parque Bicentenario 3251, Talca",2639233,94330125,D,Meléndez Moraga,Oscar Boris,"pasaje 22 norte a Parque Bicentenario 3251, Talca",5.699433e+10,2639233,male
3487,"GALLEGOS LOPEZ, HECTOR RICARDO",crisolelectricidad@gmail.com,9227339-3,"CONCEPCION 31, POBL. VALDIVIA MANZANA K, Vina...",NaN,950167335,B,GALLEGOS LOPEZ,HECTOR RICARDO,"CONCEPCION 31, POBL. VALDIVIA MANZANA K, Vina...",5.695017e+10,0,male
1665,"Soto Rodríguez , Andrés Alejandro",andres.ing.elec@gmail.com,16645144-2,"Visviri 4037, c, poblacion nueva esperanza, Macul",02-22762080,09-62137289,A,Soto Rodríguez,Andrés Alejandro,"Visviri 4037, c, poblacion nueva esperanza, Macul",5.696214e+10,222762080,male
4277,"Mora Jara , Pedro Eugenio",pmoraj@amproelec.tie.cl,10409747-2,"Santa Maria 222, -, -, Buin",228213239,98208565,B,Mora Jara,Pedro Eugenio,"Santa Maria 222, -, -, Buin",5.699821e+10,228213239,male
985,"VASQUEZ GILABERT, CARLOS HUMBERTO",cvg269@hotmail.com,6765154-5,"san joaquin de Los Mayos 680 Casa 3, Machalí",NaN,092360707,1,VASQUEZ GILABERT,CARLOS HUMBERTO,"san joaquin de Los Mayos 680 Casa 3, Machalí",5.699236e+10,0,male
3123,"Cabrera Curilef , Juan Eduardo",juancabreraelec@gmail.com,14392618-4,"calle uno 395, Villa Belen, Penco",241425415,984284851,D,Cabrera Curilef,Juan Eduardo,"calle uno 395, Villa Belen, Penco",5.698428e+10,241425415,male
9045,"Madriaga Astudillo , Esteban Jesús",esteban.madriaga.astudillo@gmail.com,17389946-7,"Av las Torres 326, Valle lo Campino, Quilicura",0224048188,82257229,B,Madriaga Astudillo,Esteban Jesús,"Av las Torres 326, Valle lo Campino, Quilicura",5.698226e+10,224048188,male


## territory functions

In [39]:
#transform functions

#extract city
def extract_last_col(dir:str)->str|None:
  dir_list = list(reversed(dir.split(',')))
  if len(dir_list)==0:
    return None
  return dir_list[0]


#name case format
import re

def namecase(string:str)->str:
  s_list = re.split('\s+',string.strip())
  for index, word in enumerate(s_list):
    s_list[index] = word[0].upper() + word[1:].lower()
  return ' '.join(s_list)

def setType(input_type:str)->str:
  if input_type == '1 y 4':
    return 'gas'
  if input_type.isnumeric():
    return 'gas'
  else:
    return 'eli'

def getAge(rut:str)->int:
  PENDANT = 3.3363697569700348e-06
  INTERCEPT = 1932.2573852507373
  rut_compose:list[str] = rut.split('-')
  return int(int(rut_compose[0])*PENDANT+INTERCEPT)


  return 0

#Get Territory
from difflib import SequenceMatcher
DPA_PATH = '/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/CUT2018.xls'
DPA = pd.read_excel(DPA_PATH)
from pandas import Series

class Territory(Enum):
  CITY='city'
  PROVINCE='province'
  REGION='region'

def getTerritory (query:str,req:Territory=Territory.CITY,res:Territory=Territory.REGION)->Series:
  #generate a serie with ratio similatities value
  ratio_series:DataFrame = DPA[req.value].apply(lambda req: SequenceMatcher(None,query,req).ratio())
  #find the most simliar ratio an get this index
  most_similar_index = ratio_series.idxmax()
  #extract row index
  result:Series = DPA.iloc[most_similar_index]
  #return value require, like city, region or province
  return result[res.value]

import math
def normalizePhone(input:int|None)->int|None:

  if len(str(input)) == len('56948611376'):
    return input
  if len(str(input)) == len('948611376'):
    return int('56'+str(input))
  if len(str(input)) == len('48611376'):
    return int('569'+str(input))
  return None

In [40]:
getTerritory('Viña del ',res=Territory.CITY)

'Viña del Mar'

In [41]:
#Rename columns
result['city'] = result['address'].map(extract_last_col)
result['city'] = result['city'].apply(lambda c:getTerritory(c,res=Territory.CITY))
result['region'] = result['city'].apply(lambda c:getTerritory(c,res=Territory.REGION))
result['birth'] = result['RUT'].apply(getAge)
result['email'] = result['CORREO ELECTRÓNICO'].str.lower()

#set licence class
result['service'] = result['LICENCIA CLASE'].map(setType)


result.sample(10)

,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,surname,name,address,movil,phone,gender,city,region,birth,email,service
109,"Ruiz Méndez , Ronnie Agustín",gaslinltda@gmail.com,9172943-1,"COLO COLO 74, Linares",732217792,99780208,1,Ruiz Méndez,Ronnie Agustín,"COLO COLO 74, Linares",5.699978e+10,732217792,male,Linares,Maule,1962,gaslinltda@gmail.com,gas
8534,"Copa Vilca , Héctor Carlos",copavilca@gmail.com,17011487-6,"San miguel 464, Valle de Azapa, Arica",NaN,961204130,B,Copa Vilca,Héctor Carlos,"San miguel 464, Valle de Azapa, Arica",5.696120e+10,0,male,Arica,Arica y Parinacota,1989,copavilca@gmail.com,eli
2123,"Espinoza Muñoz , Osvaldo Enrique",oespinoza2021@gmail.com,17716183-7,"Chomedahue 0, 0, Santa Cruz",NaN,950734843,B,Espinoza Muñoz,Osvaldo Enrique,"Chomedahue 0, 0, Santa Cruz",5.695073e+10,0,male,Santa Cruz,Libertador General Bernardo O'Higgins,1991,oespinoza2021@gmail.com,eli
8164,"Urquieta Ossandón , Roberto Carlos",rurquieta74@gmail.com,12842993-K,"flamencos 570, poblacion Bosque San Carlos, Co...",512622981,95356338,B,Urquieta Ossandón,Roberto Carlos,"flamencos 570, poblacion Bosque San Carlos, Co...",5.699536e+10,512622981,male,Coquimbo,Coquimbo,1975,rurquieta74@gmail.com,eli
2223,"Bejares Escorcia , Juan Israel",juan.bejares1989@gmail.com,17213752-0,"mallorca 568, ex-Villa Alemana",0322117337,56959494111,B,Bejares Escorcia,Juan Israel,"mallorca 568, ex-Villa Alemana",5.695949e+10,322117337,male,Villa Alemana,Valparaíso,1989,juan.bejares1989@gmail.com,eli
5592,"Medina Fernández , Felipe Esteban",felipe.medina.f28@gmail.com,19339026-9,"Nirivilo 705, Villa Rukan, Vina del Mar",NaN,931861863,B,Medina Fernández,Felipe Esteban,"Nirivilo 705, Villa Rukan, Vina del Mar",5.693186e+10,0,male,Viña del Mar,Valparaíso,1996,felipe.medina.f28@gmail.com,eli
4641,"Aravena Pinilla , Ricardo Andres",ricardo.pinilla01@gmail.com,17756435-4,"carrera 215, Chillan",945084222,945084222,A,Aravena Pinilla,Ricardo Andres,"carrera 215, Chillan",5.694508e+10,945084222,male,Chillán,Ñuble,1991,ricardo.pinilla01@gmail.com,eli
9055,"Maureira Calderón , Javier Andrés",maureira.915@gmail.com,17936549-9,"General Velasquez 1494, Antofagasta",NaN,9-98339821,A,Maureira Calderón,Javier Andrés,"General Velasquez 1494, Antofagasta",5.699834e+10,0,male,Antofagasta,Antofagasta,1992,maureira.915@gmail.com,eli
2903,"Gallardo Fuentes , Claudio Ignacio",cgallardo@buin.cl,19721385-K,"Nuestra Ilusion 0164, Buin, Buin",993136653,993136653,A,Gallardo Fuentes,Claudio Ignacio,"Nuestra Ilusion 0164, Buin, Buin",5.699314e+10,993136653,male,Buin,Metropolitana de Santiago,1998,cgallardo@buin.cl,eli
1238,"Novoa Jorquera , Jorge Alejandro",jnovoajorquera@gmail.com,13606662-5,"Camino Publico Castro 3161, Sector Ten Ten, Ch...",NaN,09-56382347,1,Novoa Jorquera,Jorge Alejandro,"Camino Publico Castro 3161, Sector Ten Ten, Ch...",5.695638e+10,0,male,Castro,Los Lagos,1977,jnovoajorquera@gmail.com,gas


In [42]:
#content with Name Format
for field in ['name','surname','address']:
  result[field] = result[field].map(namecase)

result.sample(10)

,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,surname,name,address,movil,phone,gender,city,region,birth,email,service
2203,"Méndez Manríquez , Fernando Mauricio",mendezelectricidad@gmail.com,9814353-K,"MONTE GRANDE 507, La Reina",NaN,981981034,B,Méndez Manríquez,Fernando Mauricio,"Monte Grande 507, La Reina",5.698198e+10,0,male,La Reina,Metropolitana de Santiago,1965,mendezelectricidad@gmail.com,eli
9816,"Jara Bravo , Elías Juan",elias.jara93@gmail.com,18342747-4,"Alqueria Sur 898, Linares",NaN,993165412,B,Jara Bravo,Elías Juan,"Alqueria Sur 898, Linares",5.699317e+10,0,male,Linares,Maule,1993,elias.jara93@gmail.com,eli
2093,"Ortega Rueda , Dairo Jose",dairor57@yahoo.es,21719979-4,"CARMEN 390, 209, Santiago",NaN,994786726,C,Ortega Rueda,Dairo Jose,"Carmen 390, 209, Santiago",5.699479e+10,0,male,Santiago,Metropolitana de Santiago,2004,dairor57@yahoo.es,eli
4844,"Román Beltrán , Sergio Luis",sergioromanbeltran06@gmail.com,16191981-0,"antartica 238, poblacion hermanos carrera, Val...",NaN,940997651,B,Román Beltrán,Sergio Luis,"Antartica 238, Poblacion Hermanos Carrera, Val...",5.694100e+10,0,male,Vallenar,Atacama,1986,sergioromanbeltran06@gmail.com,eli
9559,"Santander Ibacache , Pedro Bernardo",pedrobernardsantander@gmail.com,17126653-K,"puren 476, z. ex-Chillán",NaN,56995988018,B,Santander Ibacache,Pedro Bernardo,"Puren 476, Z. Ex-chillán",5.699599e+10,0,male,Chillán,Ñuble,1989,pedrobernardsantander@gmail.com,eli
220,"Gálvez Casanova , Carlos Segundo",carlos.galvez.casanova@gmail.com,12506623-2,"pasaje ocho 5521, Pedro Aguirre Cerda",29268709,65775010,B,Gálvez Casanova,Carlos Segundo,"Pasaje Ocho 5521, Pedro Aguirre Cerda",5.696578e+10,29268709,male,Pedro Aguirre Cerda,Metropolitana de Santiago,1973,carlos.galvez.casanova@gmail.com,eli
4599,"Jaime Añasco , Oscar Eduardo",edoluz@gmail.com,10720539-K,"TEODORO ZENTENO 140, POBL. DIEGO PORTALES, San...",NaN,96708526,B,Jaime Añasco,Oscar Eduardo,"Teodoro Zenteno 140, Pobl. Diego Portales, San...",5.699671e+10,0,male,San Esteban,Valparaíso,1968,edoluz@gmail.com,eli
3043,"Ramos Armijo , Octavio Andrés",o.ramosarmijo@gmail.com,13266025-5,"Walker Martínez 1000, 31-B, La Florida",NaN,990854004,D,Ramos Armijo,Octavio Andrés,"Walker Martínez 1000, 31-b, La Florida",5.699085e+10,0,male,La Florida,Metropolitana de Santiago,1976,o.ramosarmijo@gmail.com,eli
7286,"Herrera Núñez , Pablo Javier",pabloherreran@gmail.com,14563530-6,"Exequiel Fernandez 5221, Macul",NaN,966097860,A,Herrera Núñez,Pablo Javier,"Exequiel Fernandez 5221, Macul",5.696610e+10,0,male,Macul,Metropolitana de Santiago,1980,pabloherreran@gmail.com,eli
6745,"Orellana Leaño , Daniel Rolando",dorellanal@yahoo.com,9910872-K,"Av. Argentina 33, 302, Antofagasta",485389,0978520458,A,Orellana Leaño,Daniel Rolando,"Av. Argentina 33, 302, Antofagasta",5.697852e+10,485389,male,Antofagasta,Antofagasta,1965,dorellanal@yahoo.com,eli


In [43]:
#remove unnecesary data
result = result.drop(['NOMBRE','CORREO ELECTRÓNICO','DIRECCIÓN','CELULAR','TELÉFONO'],axis=1)


In [44]:
result = result.rename(columns ={'LICENCIA CLASE':'licence','RUT':'rut'})
result.sample(10)

,rut,licence,surname,name,address,movil,phone,gender,city,region,birth,email,service
6607,7659206-3,A,Sandoval Pérez,Juan Alejandro,"Alonso De Ercilla 62, Las Salinas, Talcahuano",5.694408e+10,2592537,male,Talcahuano,Biobío,1957,j62sandoval@gmail.com,eli
878,16909061-0,B,Díaz Miranda,Javier Andrés,"Albano 161, 506, La Cisterna",5.699655e+10,96551430,male,La Cisterna,Metropolitana de Santiago,1988,j.diaz@solgelec.cl,eli
8823,17026024-4,A,Lizama Aldana,Alexander Brian,"Ecuador 5211, Lo Prado",5.695404e+10,954044391,male,Lo Prado,Metropolitana de Santiago,1989,alizama.aldana@gmail.com,eli
2723,9062256-0,A,Buccioni Reyes,Ernesto Mauricio,"Los Tamarindos 2439, Pard. 33 Concha Y Toro Es...",5.699321e+10,87788663,male,Providencia,Metropolitana de Santiago,1962,buccionireyes@gmail.com,eli
831,12403824-3,3,Martin Arancibia,Carolina Andrea,"Volcan Yelcho 3110, La Florida",5.697780e+10,0,female,La Florida,Metropolitana de Santiago,1973,accmconstruccion@gmail.com,gas
8802,13588401-4,A,Castillo Carrasco,Carlos Daniel,"Wuinkul Huinchulco 1165, Lomas De Maquehue, Pa...",5.696170e+10,0,male,Padre Las Casas,La Araucanía,1977,carloscastilloelec@gmail.com,eli
2006,19726869-7,B,Díaz Fernández,Enzo Gabriel,"Bellotas 42, Nogales",5.694524e+10,332280565,male,Nogales,Valparaíso,1998,enzodiazfer19@hotmail.com,eli
2240,16166138-4,B,Meza Arroyo,Ricardo Antonio,"Titiritero Jose Pepe Ferrari 0416, Villa Las T...",5.697192e+10,0,male,San Fernando,Libertador General Bernardo O'Higgins,1986,ricardo_1805@hotmail.com,eli
4797,15807030-8,A,Soto Cea,Víctor Raúl,"Camino Real 2080, Vina Del Mar",5.697136e+10,0,male,Viña del Mar,Valparaíso,1984,vraulsoto@gmail.com,eli
713,17326090-3,A,Aguayo Poblete,Guillermo Andrés,"Mar Del Norte 9, 22, Pasaje 9 Casa 22, Quintero",5.695015e+10,322935359,male,Quintero,Valparaíso,1990,g.aguayopoblete@gmail.com,eli


In [45]:
result_path = f'{PATH_GDRIVE}/sec_result.xlsx'
result.to_excel(result_path)

# Optional filter

In [ ]:
reload_path = f'{PATH_GDRIVE}/sec_result.xlsx'
reloaded = pd.read_excel(reload_path)

Filter regional (*optional*)

In [ ]:
result_path_v = f'{PATH_GDRIVE}/sec_result_v.xlsx'
reloaded.loc[reload['region']=='Valparaíso'].to_excel(result_path_v)

In [ ]:
reloaded.sample(5)

,Unnamed: 0,email,rut,address,phone,licence,Surname,names,gender,movil,city,region,birth,service
3346,3346,emauricio.rojas@gmail.com,16737596-0,"Las Palmas 65, Villa San Judas Tadeo, Malloa",976053172,B,Rojas Mañán,Edwin Mauricio,male,5.697605e+10,Malloa,Libertador General Bernardo O'Higgins,1988,eli
9418,9418,patriciomolina019@gmail.com,18207270-2,"Ramirez 2297, Villa Abedules, La Union",967615944,A,Molina Reyes,Patricio Antonio,male,5.696762e+10,La Unión,Los Ríos,1993,eli
1971,1971,moi.solar29@gmail.com,18326150-9,"Cerro Nevado 787, Quilicura",NaN,D,Solar Zúñiga,Moisés Alejandro,male,5.693011e+10,Quilicura,Metropolitana de Santiago,1993,eli
9621,9621,bonifaciocuri@gmail.com,16464975-K,"Poeta Juan Luis Martínez 1758, Santa María De ...",NaN,B,Maldonado Maldonado,Denis Marcelo,male,5.699843e+10,Colina,Metropolitana de Santiago,1987,eli
409,409,riego.agric@gmail.com,10791345-9,"Esteban Grlica 632, Villa Las Salinas Sindempa...",051289194,B,Rojas Cisternas,Jorge Wilson,male,5.696850e+10,Coquimbo,Coquimbo,1968,eli


In [ ]:
reloaded.to_excel(reload_path)

#Bulk load to firebase

In [46]:
#firebase
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
#service as account
credential_path='/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/firebase.json'
credential = credentials.Certificate(credential_path)
firebase_admin.initialize_app(credential)
db = firestore.client()

ValueError: ignored

In [47]:
#sec_result reload
source_path = '/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/sec_result.xlsx'
source = pd.read_excel(source_path)
source.sample(8)


,Unnamed: 0,rut,licence,surname,name,address,movil,phone,gender,city,region,birth,email,service
10978,1097,15374132-8,3,Vera Rodríguez,Carolina Angélica,"Avenida Universidad 630, 104, Curauma, Valparaíso",5.696407e+10,0,female,Valparaíso,Valparaíso,1983,caro.vera.ro@gmail.com,gas
1933,1933,13621380-6,B,Flores Cortés,José Rodrigo,"Sta Juana 7640, San Ignacio",5.699389e+10,0,male,San Ignacio,Ñuble,1977,jflores.c@hotmail.com,eli
10275,394,9645890-8,3,Reinoso Flores,Luis Americo,"Hamburgo 3501, Villa Cimm, Calama",5.697951e+10,552345593,male,Calama,Antofagasta,1964,luis.reinoso.f1@gmail.com,gas
1369,1369,13516121-7,D,Ramírez Lagos,Pablo César,"Valle Del Arcoiris 0349, Padre Las Casas",5.698532e+10,452333940,male,Padre Las Casas,La Araucanía,1977,pramirez392@gmail.com,eli
2693,2693,15757220-2,D,Coronado Acuña,Oscar Reinaldo,"Oye Río Bueno 1294, Villa Ríos Del Sur, Chilla...",5.694242e+10,217113,male,Chillán Viejo,Ñuble,1984,ocoronadoa@gmail.com,eli
3654,3654,14346673-6,B,Galleguillos Torres,Lester Andrés,"Edison Meruane 629, Terranoble, San Felipe",5.699821e+10,0,male,San Felipe,Valparaíso,1980,lester@live.cl,eli
10811,930,16975663-5,2,Cárdenas Lepio,Paulina Soledad,"Ruiz 175, Vina Del Mar",5.695422e+10,0,female,Viña del Mar,Valparaíso,1988,paulysole@hotmail.com,gas
653,653,13329897-5,A,Calderón Arancibia,Wilson Eduardo,"Angela Romero Gutiérrez 997, Santuario De Peñu...",5.696210e+10,0,male,Coquimbo,Coquimbo,1976,w.calderona@gmail.com,eli


In [48]:
source = source.replace({np.nan:None})
source['phone'] = source['phone'].replace({0:None})
source.sample(5)

,Unnamed: 0,rut,licence,surname,name,address,movil,phone,gender,city,region,birth,email,service
4884,4884,16274881-5,D,Villagra Gatica,Oscar Andrés,"Aurelio Gonzalez Sn, Marchihue",56959793032.0,959793032,male,Marchihue,Libertador General Bernardo O'Higgins,1986,tmservicioselectricos@gmail.com,eli
1274,1274,10606207-2,A,Vidal Acuña,Jaime Michel Alban,"Colicheo 1846, Las Condes",56998630587.0,None,male,Las Condes,Metropolitana de Santiago,1967,mvidal@aqua-elec.cl,eli
11662,1781,17394620-1,3,Arriagada Urrutia,Francisco Andrés,"San Martín 3267, Chiguayante",56957808002.0,None,male,Chiguayante,Biobío,1990,fcoarriaga12@gmail.com,gas
783,783,16243523-K,A,Candia Calvo,Hernan Alejandro,"Adolfo Lopez Mateo 02320, Lo Espejo",56987220126.0,None,male,Lo Espejo,Metropolitana de Santiago,1986,alejandro.candia.c@gmail.com,eli
2494,2494,14205820-0,A,Illanes Benavides,Eduardo Alexis,"Vergara 559, 302, Santiago",56963473337.0,None,male,Santiago,Metropolitana de Santiago,1979,eduillanes@gmail.com,eli


In [49]:
#database references
providers_collection = db.collection('energy_providers').document('source_providers').collection('providers')

In [50]:
from pytz import timezone
from traitlets.traitlets import Dict
import datetime
#document converter
def provider_converter(it):
  timestamp=datetime.datetime.now(tz=timezone('America/Santiago'))

  return {
      "rut":it.rut,
      "personal":{"name":it.name,"surname":it.surname,"birth":it.birth,"gender":it.gender},
      "license":{"category":it.licence,"service":it.service},
      "address":{"street":it.address,"city":it.city,"region":it.region},
      "contact":{
          "phone": int(it.phone) if it.phone!=None else None ,
          "movil":int(it.movil) if it.movil!=None else None ,
          "email":it.email
          },
      "update_at":timestamp,
  }



#Bulk upload, test 2 times, and test again, and after go in peace
### with IF EXISTS, for avoid duplication

In [ ]:
from google.cloud.firestore_v1.base_query import FieldFilter
#load
sized = source
for tech in sized.itertuples():
  #fetch if exists
  docs = providers_collection.where(filter=FieldFilter('rut','==',tech.rut)).stream()
  docs_list = list(docs)
  #if exist 1 update
  if len(docs_list) == 1:
    put_doc = docs_list[0]
    providers_collection.document(put_doc.id).set(provider_converter(tech),merge=True)
    print('updated',put_doc.id)
  #if not with 0 create
  if len(docs_list) == 0:
    post_doc = providers_collection.document()
    post_doc.set(provider_converter(tech),merge=True)
    print('created',post_doc.id)

NameError: ignored

# DANGEG: with duplication
### handle with careful

In [51]:
from google.cloud.firestore_v1.base_query import FieldFilter
#load
sized = source
for tech in sized.itertuples():
  post_doc = providers_collection.document()
  post_doc.set(provider_converter(tech),merge=True)
  print('created',post_doc.id)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
created y3akjshxgKcC5kWxOszE
created dno9X7rMQJNr1x0rGcdJ
created Cyi1x3F7gwzcZL1D6t8m
created vltSdm6F1nv6SpQ3AigF
created 8W4P4z78ZxCIg01g05qj
created uyTjmbwC1VOpf1lJ6lH5
created DCcwktZyG2yKKBy1Dv92
created HRgeEyp726At7sppjxkX
created Qb9AA5TwnUPofYg2enCG
created 7qHzt5Uv28qPs0zFTZ5k
created 99cWxyyj31rgAeVGQ7s5
created PJAheNf0ul3veQ4o7coh
created YkiIS7i3RVNUiLMTU5Xa
created UPxCa8GUPF12rQtVCgDY
created WhM7UwUWJJQLRuTs10ZG
created J1WmYetg2aKjVrrXefXp
created 949W3Xw8A5zCrWbKq6ey
created qEYGBGWhIesd4FxOQH4k
created s1MpY857vtnylLXvdfgE
created 0lquJ2DwmhbMUpQQTVO1
created BZjvNfM7I3TZudqmb4Jd
created 3TLnEN4VKV7ZZwn1zfRn
created w3DGuqrxMCLN3cll1b0z
created d9Q9CZLKWhPo3mJLD5Xt
created NuPYOB1fHpPrg4yQXA7J
created fyeIRWMuUiDJtE8Jspu5
created dCegkiYDa5Vs3BE08DN5
created vclaST1pBc8J4xdb2iRf
created NkQrTANbHxzzMSgxOsss
created rJvEEUPaQHKnAMqFCm75
created 5H9eJxWvnyNWmaOPXsBL
created ULvl9pZpVHjG8ZnCQAT5
creat